In [2]:
import GPy as gp
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
import torchvision
import torch
import torch.nn as nn
import torch.optim as optim
import signal
import IPython

warning in stationary: failed to import cython module: falling back to numpy
warning in coregionalize: failed to import cython module: falling back to numpy
warning in choleskies: failed to import cython module: falling back to numpy


In [3]:
# load the standard mnist dataset
mnist_train = torchvision.datasets.MNIST(
    root = 'pytorch-data/',  # where to put the files
    download = True,         # if files aren't here, download them
    train = True,            # whether to import the test or the train subset
    # PyTorch uses PyTorch tensors internally, not numpy arrays, so convert them.
    transform = torchvision.transforms.ToTensor()
)
mnist_batched = torch.utils.data.DataLoader(mnist_train, batch_size=100)

In [15]:
#interruptable class can be useful for model training
class Interruptable():
    class Breakout(Exception):
        pass
    def __init__(self):
        self.interrupted = False
        self.orig_handler = None
    def __enter__(self):
        self.orig_handler = signal.getsignal(signal.SIGINT)
        signal.signal(signal.SIGINT, self.handle)
        return self.check
    def __exit__(self, exc_type, exc_val, exc_tb):
        signal.signal(signal.SIGINT, self.orig_handler)
        if exc_type == Interruptable.Breakout:
            print(' stopped')
            return True
        return False
    def handle(self, signal, frame):
        if self.interrupted:
            self.orig_handler(signal, frame)
        print('Interrupting ...', end='')
        self.interrupted = True
    def check(self):
        if self.interrupted:
            raise Interruptable.Breakout
            
def enumerate_cycle(g, shuffle=True):
    epoch = 0
    while True:
        if shuffle:
            for i,j in enumerate(np.random.permutation(len(g))):
                yield (epoch,i), g[j]
        else:
            for i,x in enumerate(g):
                yield (epoch,i), x
        epoch = epoch + 1

In [4]:
# based on the classification model used in lecture
class mnist_nn_model(nn.Module):
    def __init__(self):
        # just need to define it the other way, and then can just return the outputs from any given layer in a similar way to classify
        super().__init__()
        self.conv1 = nn.Conv2d(1,32,3,1)
        self.conv2 = nn.Conv2d(32,64,3,1)
        self.pool = nn.MaxPool2d(2)
        self.drop1 = nn.Dropout2d(.25)
        self.flat = nn.Flatten(1)
        self.lin1 = nn.Linear(9216,128)
        self.drop2 = nn.Dropout2d(.25)
        self.lin2 = nn.Linear(128,16)
        self.lin3 = nn.Linear(16,10)
    
    def forward(self,x,y):
        # log likelihood for a batch of data
        return - nn.functional.cross_entropy(self.f(x), y, reduction='none')
    
    def last_layer(self, x):
        # computing in parts so it is easy to pull out different internal pieces
        pooled = self.pool(nn.functional.relu(self.conv2(nn.functional.relu(self.conv1(x)))))
        lin1 = self.lin1(self.flat(self.drop1(nn.functional.relu(pooled))))
        lin2 = self.lin2(self.drop2(nn.functional.relu(lin1)))
        return lin2
    
    def f(self, x):
        return self.lin3(nn.functional.relu(self.last_layer(x)))
    def classify(self,x):
        # class probabilities for a single data point
        q = self.f(torch.as_tensor(x)[None,...])[0]
        return nn.functional.softmax(q, dim=0)

In [5]:
model = mnist_nn_model()
optimizer = optim.Adam(model.parameters())
model.train(mode = True)

mnist_nn_model(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop1): Dropout2d(p=0.25, inplace=False)
  (flat): Flatten(start_dim=1, end_dim=-1)
  (lin1): Linear(in_features=9216, out_features=128, bias=True)
  (drop2): Dropout2d(p=0.25, inplace=False)
  (lin2): Linear(in_features=128, out_features=16, bias=True)
  (lin3): Linear(in_features=16, out_features=10, bias=True)
)

In [76]:
iter_mnist = enumerate_cycle(mnist_batched, shuffle=False)
with Interruptable() as check_interrupted:
    for (epoch,batch_num),(imgs,lbls) in iter_mnist:
        check_interrupted()
        optimizer.zero_grad()
        loglik = model(imgs, lbls)
        e = - torch.mean(loglik)
        e.backward()
        optimizer.step()

        if batch_num % 25 == 0:
            IPython.display.clear_output(wait=True)
            print(f'epoch={epoch} batch={batch_num}/{len(mnist_batched)} loglik={-e.item()}')
    epoch += 1

epoch=8 batch=475/600 loglik=-0.013399560004472733
Interrupting ... stopped


In [6]:
model.load_state_dict(torch.load('cnn_classifier.pt'))

<All keys matched successfully>

Might be a good idea to try to figure out the maximum loglikelihood at some point. Not really sure how I would do this, but it usually seems to be a good idea in this class. I kinda want to just guess it's zero but who knows 

In [77]:
torch.save(model.state_dict(), 'cnn_classifier.pt')

In [7]:
model.train(mode=False)

mnist_nn_model(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop1): Dropout2d(p=0.25, inplace=False)
  (flat): Flatten(start_dim=1, end_dim=-1)
  (lin1): Linear(in_features=9216, out_features=128, bias=True)
  (drop2): Dropout2d(p=0.25, inplace=False)
  (lin2): Linear(in_features=128, out_features=16, bias=True)
  (lin3): Linear(in_features=16, out_features=10, bias=True)
)

In [8]:
X_bin = torch.stack([img for img,lbl in mnist_train if lbl==0 or lbl ==1])

In [11]:
features = model.last_layer(X_bin)

In [9]:
Y_bin = [lbl for img,lbl in mnist_train if lbl == 0 or lbl == 1]

In [19]:
pd.DataFrame(features.detach().numpy()).to_csv('X_binary.csv')
pd.DataFrame(Y_bin).to_csv('Y_binary.csv')

In [24]:
X = features.detach().numpy()
Y = np.array(Y)

In [9]:
X.reshape(1,-1)

NameError: name 'X' is not defined

In [10]:
# save all of these features to a csv file, however that's gonna require internet of some type. Once I have all that, this notebook should be done.
# this will be a binary dataset first of features (should have 16) and the label either 0 or 1. 
# still need to consider if it's better to do this all based on the last layer (that sigmoid layer is applied to), or the last "feature" layer right above that.

In [10]:
import requests, io
import scipy.io

In [47]:
r = requests.get("https://www.cl.cam.ac.uk/teaching/2122/DataSci/data/nmnist/mnist-with-awgn.mat")

with io.BytesIO(r.content) as f:
    data = scipy.io.loadmat(f)
    

In [48]:
wn_train_x = data['train_x'].reshape(60000,1,28,28)
wn_train_y = data['train_y']

In [49]:
decoded = np.argmax(wn_train_y, axis=1)
wn_y_bin = np.array([lbl for lbl in decoded if lbl == 0 or lbl == 1])
wn_x_bin = torch.stack([torch.Tensor(img)/255 for i,img in enumerate(wn_train_x) if decoded[i]==0 or decoded[i]==1])

In [50]:
features = model.last_layer(wn_x_bin)

In [51]:
pd.DataFrame(features.detach().numpy()).to_csv('X_binary_wn.csv')
pd.DataFrame(wn_y_bin).to_csv('Y_binary_wn.csv')

In [46]:
r = requests.get("https://www.cl.cam.ac.uk/teaching/2122/DataSci/data/nmnist/mnist-with-motion-blur.mat")

with io.BytesIO(r.content) as f:
    data = scipy.io.loadmat(f)
    mb_train_x = data['train_x'].reshape(60000,1,28,28)
    mb_train_y = data['train_y']
    decoded = np.argmax(mb_train_y, axis=1)
    mb_y_bin = np.array([lbl for lbl in decoded if lbl == 0 or lbl == 1])
    mb_x_bin = torch.stack([torch.Tensor(img)/255 for i,img in enumerate(mb_train_x) if decoded[i]==0 or decoded[i]==1])
features = model.last_layer(mb_x_bin)
pd.DataFrame(features.detach().numpy()).to_csv('X_binary_mb.csv')
pd.DataFrame(mb_y_bin).to_csv('Y_binary_mb.csv')

In [52]:
r = requests.get("https://www.cl.cam.ac.uk/teaching/2122/DataSci/data/nmnist/mnist-with-reduced-contrast-and-awgn.mat")

with io.BytesIO(r.content) as f:
    data = scipy.io.loadmat(f)
    rc_train_x = data['train_x'].reshape(60000,1,28,28)
    rc_train_y = data['train_y']
    decoded = np.argmax(rc_train_y, axis=1)
    rc_y_bin = np.array([lbl for lbl in decoded if lbl == 0 or lbl == 1])
    rc_x_bin = torch.stack([torch.Tensor(img)/255 for i,img in enumerate(rc_train_x) if decoded[i]==0 or decoded[i]==1])
features = model.last_layer(rc_x_bin)
pd.DataFrame(features.detach().numpy()).to_csv('X_binary_rc.csv')
pd.DataFrame(rc_y_bin).to_csv('Y_binary_rc.csv')

In [38]:
features = model.last_layer(X_bin)

In [45]:
X_bin[0].max()

tensor(1.)

In [11]:
from cleverhans.torch.attacks.projected_gradient_descent import projected_gradient_descent
from cleverhans.torch.attacks.fast_gradient_method import fast_gradient_method
from absl import app, flags




In [17]:
X_pgd = projected_gradient_descent(model.f, X_bin[:1000], .2, .01, 40, np.inf)

In [18]:
features = model.last_layer(X_pgd)

In [19]:
pd.DataFrame(features.detach().numpy()).to_csv('X_binary_pgd.csv')
pd.DataFrame(Y_bin[:1000]).to_csv('Y_binary_pgd.csv')

In [20]:
X_fgm = fast_gradient_method(model.f, X_bin[:1000], .2, np.inf)
features = model.last_layer(X_fgm)
pd.DataFrame(features.detach().numpy()).to_csv('X_binary_fgm.csv')
pd.DataFrame(Y_bin[:1000]).to_csv('Y_binary_fgm.csv')